## Lab 2 pt 1 - PRISM

#### Author: Jacob Harris

The goal of this notebook is to:

1. Download the annual 30-Year Normals .bil files for precipitation from PRISM
2. Convert the data into a spacetime cube and export it to disk
3. Export an animation of the timeseries

**Disclaimer**: Some parts of this code were created using ChatGPT. I also looked at documentation from Esri for certain ArcPy functions.

In [27]:
import requests
import zipfile
import os
import arcpy

In [28]:
# Send HTTP GET request to get Web API data
response = requests.get(r'https://prism.oregonstate.edu/fetchData.php?type=all_bil&kind=normals&spatial=4km&elem=ppt&temporal=annual')
zip_path = r'C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\PRISM\30yearppt.zip'
if response.status_code == 200:
    with open (zip_path, 'wb') as file:
        file.write(response.content)
        
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall (r'C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\PRISM\BIL')

In [29]:
# Set file paths
base_dir = r'C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\PRISM'
folder_path = base_dir + '\BIL'
file_list = os.listdir(folder_path)

for filename in file_list:
    if "annual" in filename.lower():  # Convert to lowercase for case-insensitive matching
        file_path = os.path.join(folder_path, filename)
        try:
            os.remove(file_path)
            print(f"Deleted: {filename}")
        except Exception as e:
            print(f"Error deleting {filename}: {e}")

Deleted: PRISM_ppt_30yr_normal_4kmM4_annual_bil.bil
Deleted: PRISM_ppt_30yr_normal_4kmM4_annual_bil.bil.aux.xml
Deleted: PRISM_ppt_30yr_normal_4kmM4_annual_bil.hdr
Deleted: PRISM_ppt_30yr_normal_4kmM4_annual_bil.info.txt
Deleted: PRISM_ppt_30yr_normal_4kmM4_annual_bil.prj
Deleted: PRISM_ppt_30yr_normal_4kmM4_annual_bil.stx
Deleted: PRISM_ppt_30yr_normal_4kmM4_annual_bil.xml


In [36]:
# Set file paths
bils_folder = base_dir + '\BIL'
tifs_folder = base_dir + '\\tifs'

os.makedirs(tifs_folder) # Create folder for future TIFF files

bil_files = [f for f in os.listdir(bils_folder) if f.lower().endswith('.bil')]

# Iterate through all .bil and .tif files 
for bil_file in bil_files:
    bil_path = os.path.join(bils_folder, bil_file)
    tif_file = os.path.splitext(bil_file)[0] + '.tif'
    tif_path = os.path.join(tifs_folder, tif_file)
    if not os.path.exists(tif_path):
        arcpy.conversion.RasterToOtherFormat(bil_path, tifs_folder, "TIFF") # Convert to TIFF
print('Files successfully converted to TIFF and stored in .tifs folder')

Files successfully converted to TIFF and stored in .tifs folder


In [38]:
# Shortcut for workspace directory
gdb_dir = base_dir + '\PRISM.gdb'
# Set coordinate system to NAD 1983
coordinate_system = arcpy.SpatialReference(4269)
in_workspace = gdb_dir
in_mosaicdataset_name = 'Normals_Precipiation'
# Create mosaic dataset
arcpy.management.CreateMosaicDataset(in_workspace, in_mosaicdataset_name, coordinate_system)

<Result 'C:\\Users\\jake1\\OneDrive\\Documents\\ArcGIS\\Projects\\PRISM\\PRISM.gdb\\Normals_Precipiation'>

In [39]:
# Add rasters to the mosaic dataset
in_mosaic_dataset = 'Normals_Precipiation'
input_path = tifs_folder
raster_type = 'Raster Dataset'
arcpy.management.AddRastersToMosaicDataset(in_mosaic_dataset, raster_type, input_path)

<Result 'Normals_Precipiation'>

In [40]:
# Updating the values in 'Variable' to "ppt" 
field_name = 'Variable'
expression = '"ppt"'
# Perform field calculation
arcpy.CalculateField_management(in_mosaic_dataset, field_name, expression, "PYTHON3")

<Result 'Normals_Precipiation'>

In [41]:
# Setting up time stamp
field_name = 'Timestamp'
type_field = 'DATE'
expression = 'DateAdd(Date( 1990, 12, 1), $feature.OBJECTID - 1, "month")'
# Perform field calculation: calculate date based on an offset
arcpy.management.CalculateField(in_mosaic_dataset, field_name, expression, "ARCADE", field_type = type_field)

<Result 'Normals_Precipiation'>

In [42]:
# Create multidimensional information for the mosaic database
variable_field = 'Variable'
timestamp_field = 'Timestamp'
arcpy.md.BuildMultidimensionalInfo(in_mosaic_dataset, variable_field, timestamp_field)

<Result 'Normals_Precipiation'>

In [43]:
# Create multidimensional raster layer
in_multidimensional_raster = in_mosaic_dataset
out_multidimensional_raster_layer = 'Normals_Precipitation_MultiDim_Layer'
arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster, out_multidimensional_raster_layer)

<Result 'Normals_Precipitation_MultiDim_Layer'>

In [46]:
# Create spacetime cube
output_cube = base_dir + '\normals_cube.nc'
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(out_multidimensional_raster_layer, output_cube, 'Zeros')

<Result 'C:\\Users\\jake1\\OneDrive\\Documents\\ArcGIS\\Projects\\PRISM\\ppt.nc'>

## Rendering .gif Using the Spacetime Cube

Before running the following cell to work, you must complete the following steps:
- Screenshot each frame of the Multidimensional Layer (Normals_Precipitation_MultiDIm_Layer) 
- Save all of those screenshots into a folder in your project directory named 'frames'


In [ ]:
from PIL import Image
import os

# Directory containing the PNG files
input_directory = r'C:\Users\jake1\OneDrive\Documents\ArcGIS\Projects\PRISM\frames'

# List of PNG files in the directory
png_files = [file for file in os.listdir(input_directory) if file.endswith(".png")]

# Sort the list of PNG files (if needed)
png_files.sort()

# Create a list to store the images
frames = []

# Loop through the PNG files, open and append them to the frames list
for png_file in png_files:
    image = Image.open(os.path.join(input_directory, png_file))
    frames.append(image)

# Output file name for the animated GIF
output_gif = 'normals.gif'

# Save the frames as an animated GIF
frames[0].save(output_gif, save_all=True, append_images=frames[1:], duration=500, loop=0)

print(f"Animated GIF saved as '{output_gif}'")